In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%config Completer.use_jedi = False        # For autocomplete to work!

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: Config option `use_jedi` not recognized by `IPCompleter`.
  


In [8]:
sns.set(rc={'figure.figsize':(10,6)})
sns.set_style('whitegrid')
sns.set_palette('Set2')

# Text Generation with Neural Networks

In this notebook we will create a network that can generate text, here we show it being done character by character. Very awesome write up on this here: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

We organized the process into "steps" so you can easily follow along with your own data sets.

In [9]:
%tensorflow_version 2.x

import tensorflow as tf

## Step 1: The Data

You can grab any free text you want from here: https://www.gutenberg.org/

We'll choose all of shakespeare's works (which we have already downloaded for you), mainly for two reasons:

1. Its a large corpus of text, its usually recommended you have at least a source of 1 million characters total to get realistic text generation.

2. It has a very distinctive style. Since the text data uses old style english and is formatted in the style of a stage play, it will be very obvious to us if the model is able to reproduce similar results.

In [10]:
path_to_file = "shakespeare.txt"

In [11]:
text = open(path_to_file,'r').read()

In [12]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


### Understanding unique characters


In [13]:
sorted(set(text))

['\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '<',
 '>',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 '[',
 ']',
 '_',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|']

In [14]:
vocab = sorted(set(text))

In [15]:
len(vocab)

82

## Step 2: Text Processing

### Text Vectorization

We know a neural network can't take in the raw string data, we need to assign numbers to each character. Let's create two dictionaries that can go from numeric index to character and character to numeric index.

In [16]:
for pair in enumerate(vocab):
    print(pair)

(0, '\n')
(1, ' ')
(2, '!')
(3, '"')
(4, '&')
(5, "'")
(6, '(')
(7, ')')
(8, ',')
(9, '-')
(10, '.')
(11, '0')
(12, '1')
(13, '2')
(14, '3')
(15, '4')
(16, '5')
(17, '6')
(18, '7')
(19, '8')
(20, '9')
(21, ':')
(22, ';')
(23, '<')
(24, '>')
(25, '?')
(26, 'A')
(27, 'B')
(28, 'C')
(29, 'D')
(30, 'E')
(31, 'F')
(32, 'G')
(33, 'H')
(34, 'I')
(35, 'J')
(36, 'K')
(37, 'L')
(38, 'M')
(39, 'N')
(40, 'O')
(41, 'P')
(42, 'Q')
(43, 'R')
(44, 'S')
(45, 'T')
(46, 'U')
(47, 'V')
(48, 'W')
(49, 'X')
(50, 'Y')
(51, 'Z')
(52, '[')
(53, ']')
(54, '_')
(55, 'a')
(56, 'b')
(57, 'c')
(58, 'd')
(59, 'e')
(60, 'f')
(61, 'g')
(62, 'h')
(63, 'i')
(64, 'j')
(65, 'k')
(66, 'l')
(67, 'm')
(68, 'n')
(69, 'o')
(70, 'p')
(71, 'q')
(72, 'r')
(73, 's')
(74, 't')
(75, 'u')
(76, 'v')
(77, 'w')
(78, 'x')
(79, 'y')
(80, 'z')
(81, '|')


In [17]:
char_to_ind = {char:ind for ind,char in enumerate(vocab)}

In [18]:
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 '&': 4,
 "'": 5,
 '(': 6,
 ')': 7,
 ',': 8,
 '-': 9,
 '.': 10,
 '0': 11,
 '1': 12,
 '2': 13,
 '3': 14,
 '4': 15,
 '5': 16,
 '6': 17,
 '7': 18,
 '8': 19,
 '9': 20,
 ':': 21,
 ';': 22,
 '<': 23,
 '>': 24,
 '?': 25,
 'A': 26,
 'B': 27,
 'C': 28,
 'D': 29,
 'E': 30,
 'F': 31,
 'G': 32,
 'H': 33,
 'I': 34,
 'J': 35,
 'K': 36,
 'L': 37,
 'M': 38,
 'N': 39,
 'O': 40,
 'P': 41,
 'Q': 42,
 'R': 43,
 'S': 44,
 'T': 45,
 'U': 46,
 'V': 47,
 'W': 48,
 'X': 49,
 'Y': 50,
 'Z': 51,
 '[': 52,
 ']': 53,
 '_': 54,
 'a': 55,
 'b': 56,
 'c': 57,
 'd': 58,
 'e': 59,
 'f': 60,
 'g': 61,
 'h': 62,
 'i': 63,
 'j': 64,
 'k': 65,
 'l': 66,
 'm': 67,
 'n': 68,
 'o': 69,
 'p': 70,
 'q': 71,
 'r': 72,
 's': 73,
 't': 74,
 'u': 75,
 'v': 76,
 'w': 77,
 'x': 78,
 'y': 79,
 'z': 80,
 '|': 81}

What's the numerical index of the letter `H`?

In [19]:
char_to_ind['H']

33

And, for the other way around we simply put the `vocab` in a numpy array, so the index is the corresponding character.

In [20]:
ind_to_char = np.array(vocab)

In [21]:
ind_to_char[33]

'H'

In [22]:
encoded_text = np.array([char_to_ind[c] for c in text])

In [23]:
encoded_text[:500]

array([ 0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, 12,  0,  1,  1, 31, 72, 69, 67,  1, 60, 55, 63,
       72, 59, 73, 74,  1, 57, 72, 59, 55, 74, 75, 72, 59, 73,  1, 77, 59,
        1, 58, 59, 73, 63, 72, 59,  1, 63, 68, 57, 72, 59, 55, 73, 59,  8,
        0,  1,  1, 45, 62, 55, 74,  1, 74, 62, 59, 72, 59, 56, 79,  1, 56,
       59, 55, 75, 74, 79,  5, 73,  1, 72, 69, 73, 59,  1, 67, 63, 61, 62,
       74,  1, 68, 59, 76, 59, 72,  1, 58, 63, 59,  8,  0,  1,  1, 27, 75,
       74,  1, 55, 73,  1, 74, 62, 59,  1, 72, 63, 70, 59, 72,  1, 73, 62,
       69, 75, 66, 58,  1, 56, 79,  1, 74, 63, 67, 59,  1, 58, 59, 57, 59,
       55, 73, 59,  8,  0,  1,  1, 33, 63, 73,  1, 74, 59, 68, 58, 59, 72,
        1, 62, 59, 63, 72,  1, 67, 63, 61, 62, 74,  1, 56, 59, 55, 72,  1,
       62, 63, 73,  1, 67, 59, 67, 69, 72, 79, 21,  0,  1,  1, 27, 75, 74,
        1, 74, 62, 69, 75,  1, 57, 69, 68, 74, 72, 55, 57, 74, 59, 58,  1,
       74, 69,  1, 74, 62

In [24]:
encoded_text.shape

(1048576,)

We now have a mapping we can use to go back and forth from characters to numerics.

In [25]:
sample = text[:20]
sample

'\n                   '

In [26]:
encoded_text[:20]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Step 3: Creating Batches

Overall what we are trying to achieve is to have the model predict the next highest probability character given a historical sequence of characters. Its up to us (the user) to choose how long that historic sequence. Too short a sequence and we don't have enough information (e.g. given the letter "a" , what is the next character) , too long a sequence and training will take too long and most likely overfit to sequence characters that are irrelevant to characters farther out. While there is no correct sequence length choice, you should consider the text itself, how long normal phrases are in it, and a reasonable idea of what characters/words are relevant to each other.

In [27]:
print(text[:500])


                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
  His tender heir might bear his memory:
  But thou contracted to thine own bright eyes,
  Feed'st thy light's flame with self-substantial fuel,
  Making a famine where abundance lies,
  Thy self thy foe, to thy sweet self too cruel:
  Thou that art now the world's fresh ornament,
  And only herald to the gaudy spring,
  Within thine own bu


In [28]:
line = 'From fairest creatures we desire increase'

In [29]:
len(line)

41

In [30]:
lines = '''
From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But as the riper should by time decease,
'''

In [31]:
len(lines)

133

### Training Sequences

The actual text data will be the text sequence shifted one character forward. For example:

Sequence In: "Hello my nam"
Sequence Out: "ello my name"


We can use the `tf.data.Dataset.from_tensor_slices` function to convert a text vector into a stream of character indices.

In [32]:
seq_len = 120

In [33]:
total_num_seq = len(text) // (seq_len + 1)

In [34]:
total_num_seq

8665

In [35]:
# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [36]:
type(char_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [37]:
for item in char_dataset.take(500):
    print(item.numpy())

0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
12
0
1
1
31
72
69
67
1
60
55
63
72
59
73
74
1
57
72
59
55
74
75
72
59
73
1
77
59
1
58
59
73
63
72
59
1
63
68
57
72
59
55
73
59
8
0
1
1
45
62
55
74
1
74
62
59
72
59
56
79
1
56
59
55
75
74
79
5
73
1
72
69
73
59
1
67
63
61
62
74
1
68
59
76
59
72
1
58
63
59
8
0
1
1
27
75
74
1
55
73
1
74
62
59
1
72
63
70
59
72
1
73
62
69
75
66
58
1
56
79
1
74
63
67
59
1
58
59
57
59
55
73
59
8
0
1
1
33
63
73
1
74
59
68
58
59
72
1
62
59
63
72
1
67
63
61
62
74
1
56
59
55
72
1
62
63
73
1
67
59
67
69
72
79
21
0
1
1
27
75
74
1
74
62
69
75
1
57
69
68
74
72
55
57
74
59
58
1
74
69
1
74
62
63
68
59
1
69
77
68
1
56
72
63
61
62
74
1
59
79
59
73
8
0
1
1
31
59
59
58
5
73
74
1
74
62
79
1
66
63
61
62
74
5
73
1
60
66
55
67
59
1
77
63
74
62
1
73
59
66
60
9
73
75
56
73
74
55
68
74
63
55
66
1
60
75
59
66
8
0
1
1
38
55
65
63
68
61
1
55
1
60
55
67
63
68
59
1
77
62
59
72
59
1
55
56
75
68
58
55
68
57
59
1
66
63
59
73
8
0
1
1
45
62
79
1
73
59
66
60
1
74
62
79
1
60
69
59
8
1
74
69
1
74
62


In [38]:
for item in char_dataset.take(500):
    print(ind_to_char[item.numpy()])



 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
1


 
 
F
r
o
m
 
f
a
i
r
e
s
t
 
c
r
e
a
t
u
r
e
s
 
w
e
 
d
e
s
i
r
e
 
i
n
c
r
e
a
s
e
,


 
 
T
h
a
t
 
t
h
e
r
e
b
y
 
b
e
a
u
t
y
'
s
 
r
o
s
e
 
m
i
g
h
t
 
n
e
v
e
r
 
d
i
e
,


 
 
B
u
t
 
a
s
 
t
h
e
 
r
i
p
e
r
 
s
h
o
u
l
d
 
b
y
 
t
i
m
e
 
d
e
c
e
a
s
e
,


 
 
H
i
s
 
t
e
n
d
e
r
 
h
e
i
r
 
m
i
g
h
t
 
b
e
a
r
 
h
i
s
 
m
e
m
o
r
y
:


 
 
B
u
t
 
t
h
o
u
 
c
o
n
t
r
a
c
t
e
d
 
t
o
 
t
h
i
n
e
 
o
w
n
 
b
r
i
g
h
t
 
e
y
e
s
,


 
 
F
e
e
d
'
s
t
 
t
h
y
 
l
i
g
h
t
'
s
 
f
l
a
m
e
 
w
i
t
h
 
s
e
l
f
-
s
u
b
s
t
a
n
t
i
a
l
 
f
u
e
l
,


 
 
M
a
k
i
n
g
 
a
 
f
a
m
i
n
e
 
w
h
e
r
e
 
a
b
u
n
d
a
n
c
e
 
l
i
e
s
,


 
 
T
h
y
 
s
e
l
f
 
t
h
y
 
f
o
e
,
 
t
o
 
t
h
y
 
s
w
e
e
t
 
s
e
l
f
 
t
o
o
 
c
r
u
e
l
:


 
 
T
h
o
u
 
t
h
a
t
 
a
r
t
 
n
o
w
 
t
h
e
 
w
o
r
l
d
'
s
 
f
r
e
s
h
 
o
r
n
a
m
e
n
t
,


 
 
A
n
d
 
o
n
l
y
 
h
e
r
a
l
d
 
t
o
 
t
h
e
 
g
a
u
d
y
 
s
p
r
i
n
g
,


 
 
W
i
t
h
i
n
 
t
h
i
n
e
 
o
w
n
 
b
u


The **batch** method converts these individual character calls into sequences we can feed in as a batch. We use seq_len+1 because of zero indexing. Here is what drop_remainder means:

drop_remainder: (Optional.) A `tf.bool` scalar `tf.Tensor`, representing
    whether the last batch should be dropped in the case it has fewer than
    `batch_size` elements; the default behavior is not to drop the smaller
    batch.

In [39]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

Now that we have our sequences, we will perform the following steps for each one to create our target text sequences:

1. Grab the input text sequence
2. Assign the target text sequence as the input text sequence shifted by one step forward
3. Group them together as a tuple

In [40]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

In [41]:
dataset = sequences.map(create_seq_targets)

In [42]:
for input_txt, target_txt in dataset.take(1):
    print(input_txt.numpy())
    print("".join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    print("".join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 72 69 67  1 60 55 63 72 59 73 74  1 57 72 59 55 74 75 72 59 73
  1 77 59  1 58 59 73 63 72 59  1 63 68 57 72 59 55 73 59  8  0  1  1 45
 62 55 74  1 74 62 59 72 59 56 79  1 56 59 55 75 74 79  5 73  1 72 69 73
 59  1 67 63 61 62 74  1 68 59 76 59 72  1 58 63 59  8  0  1  1 27 75 74]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 72 69 67  1 60 55 63 72 59 73 74  1 57 72 59 55 74 75 72 59 73  1
 77 59  1 58 59 73 63 72 59  1 63 68 57 72 59 55 73 59  8  0  1  1 45 62
 55 74  1 74 62 59 72 59 56 79  1 56 59 55 75 74 79  5 73  1 72 69 73 59
  1 67 63 61 62 74  1 68 59 76 59 72  1 58 63 59  8  0  1  1 27 75 74  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


### Generating training batches

Now that we have the actual sequences, we will create the batches, we want to shuffle these sequences into a random order, so the model doesn't overfit to any section of the text, but can instead generate characters given any seed text.

In [43]:
batch_size = 128

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [44]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int64, tf.int64)>

## Step 4: Creating the Model


We will use an LSTM based model with a few extra features, including an embedding layer to start off with and **two** LSTM layers. We based this model architecture off the [DeepMoji](https://deepmoji.mit.edu/) and the original source code can be found [here](https://github.com/bfelbo/DeepMoji).

The embedding layer will serve as the input layer, which essentially creates a lookup table that maps the numbers indices of each character to a vector with "embedding dim" number of dimensions. As you can imagine, the larger this embedding size, the more complex the training. This is similar to the idea behind word2vec, where words are mapped to some n-dimensional space. Embedding before feeding straight into the LSTM usually leads to more realisitic results.

In [45]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embed_dim = 64

# Number of RNN units
rnn_neurons = 1026

Now let's create a function that easily adapts to different variables as shown above.

In [46]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GRU

[Help with the choice of loss function](https://datascience.stackexchange.com/questions/41921/sparse-categorical-crossentropy-vs-categorical-crossentropy-keras-accuracy)

### Setting up Loss Function

For our loss we will use sparse categorical crossentropy, which we can import from Keras. We will also set this as logits=True

In [47]:
def sparse_cat_loss(y_true, y_pred):
    return sparse_categorical_crossentropy(y_true,y_pred, from_logits=True)

In [48]:
def create_model(vocab_size, embed_dim, rnn_neurons, batch_size):
    
    model = Sequential()
    
    model.add(Embedding(vocab_size, embed_dim,
                        batch_input_shape=[batch_size,None]))
    
    model.add(GRU(rnn_neurons,return_sequences=True, stateful=True,
                  recurrent_initializer='glorot_uniform'))
    
    model.add(Dense(vocab_size))
    
    model.compile('adam', loss=sparse_cat_loss)
    
    return model

In [49]:
model = create_model(
            vocab_size=vocab_size,
            embed_dim=embed_dim,
            rnn_neurons=rnn_neurons,
            batch_size=batch_size)

In [50]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5248      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 82)           84214     
Total params: 3,450,638
Trainable params: 3,450,638
Non-trainable params: 0
_________________________________________________________________


## Step 5: Training the model

Let's make sure everything is ok with our model before we spend too much time training! Let's pass in a batch to confirm the model currently predicts random characters without any training.

In [51]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, "<===(batch_size, sequence_length, vocab_size)")

(128, 120, 82) <===(batch_size, sequence_length, vocab_size)


In [52]:
example_batch_predictions[0]

<tf.Tensor: shape=(120, 82), dtype=float32, numpy=
array([[-4.27961117e-03, -4.54277452e-03,  4.12639603e-03, ...,
         7.07565050e-06,  1.79764582e-03, -6.79894164e-03],
       [-1.59637444e-03, -5.13449253e-04,  7.45236687e-03, ...,
         1.51748175e-03,  1.49562271e-04, -3.80668300e-03],
       [-4.99771850e-04, -6.95040682e-03,  5.78096695e-03, ...,
         1.20473222e-03,  5.22901909e-03, -2.97621498e-03],
       ...,
       [-7.28792232e-03, -7.58012384e-03,  3.34664294e-03, ...,
         1.18032876e-04,  2.72836140e-03, -7.64672179e-03],
       [ 3.33023467e-03,  1.60021312e-03,  3.59666912e-04, ...,
         2.99664284e-03, -1.08204351e-03,  7.18554133e-04],
       [ 1.63077051e-03,  1.53752847e-03,  5.27598104e-03, ...,
         2.28313613e-03, -2.24717497e-03,  1.71120360e-03]], dtype=float32)>

In [53]:
sampled_indices = tf.random.categorical(example_batch_predictions[0],num_samples=1).numpy()

In [54]:
sampled_indices

array([[10],
       [30],
       [49],
       [70],
       [37],
       [53],
       [28],
       [59],
       [ 7],
       [75],
       [62],
       [74],
       [42],
       [20],
       [18],
       [ 7],
       [43],
       [29],
       [76],
       [53],
       [20],
       [ 7],
       [32],
       [70],
       [37],
       [24],
       [47],
       [59],
       [ 1],
       [ 4],
       [12],
       [42],
       [49],
       [ 6],
       [26],
       [48],
       [61],
       [ 7],
       [ 2],
       [26],
       [ 8],
       [42],
       [79],
       [71],
       [12],
       [66],
       [25],
       [37],
       [28],
       [ 8],
       [61],
       [58],
       [12],
       [19],
       [79],
       [15],
       [29],
       [72],
       [60],
       [77],
       [67],
       [45],
       [ 8],
       [79],
       [ 2],
       [62],
       [39],
       [42],
       [25],
       [14],
       [26],
       [21],
       [ 2],
       [73],
       [21],
       [43],
       [15],

In [55]:
sampled_indices = tf.squeeze(sampled_indices, axis=1).numpy()

In [56]:
ind_to_char[sampled_indices]

array(['.', 'E', 'X', 'p', 'L', ']', 'C', 'e', ')', 'u', 'h', 't', 'Q',
       '9', '7', ')', 'R', 'D', 'v', ']', '9', ')', 'G', 'p', 'L', '>',
       'V', 'e', ' ', '&', '1', 'Q', 'X', '(', 'A', 'W', 'g', ')', '!',
       'A', ',', 'Q', 'y', 'q', '1', 'l', '?', 'L', 'C', ',', 'g', 'd',
       '1', '8', 'y', '4', 'D', 'r', 'f', 'w', 'm', 'T', ',', 'y', '!',
       'h', 'N', 'Q', '?', '3', 'A', ':', '!', 's', ':', 'R', '4', '[',
       'E', '9', '?', '9', '&', 'a', 'V', 'z', 'F', 'V', 'T', 'L', '!',
       'D', 'b', 'a', ';', '\n', 'V', 'o', 'k', 'M', '&', 'c', 'C', 'Z',
       's', 'X', '6', '6', '9', 'F', 'q', "'", 'V', 'w', 'w', 'T', 'S',
       'k', 'm', 'G'], dtype='<U1')

In [57]:
print("Given the input seq: \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(ind_to_char[sampled_indices ]))

Given the input seq: 

d queen,
  The basest jewel will be well esteemed:
  So are those errors that in thee are seen,
  To truths translated, 


Next Char Predictions: 

.EXpL]Ce)uhtQ97)RDv]9)GpL>Ve &1QX(AWg)!A,Qyq1l?LC,gd18y4DrfwmT,y!hNQ?3A:!s:R4[E9?9&aVzFVTL!Dba;
VokM&cCZsX669Fq'VwwTSkmG


After confirming the dimensions are working, let's train our network!

In [58]:
epochs = 30

In [59]:
model.fit(dataset, epochs=epochs)

Epoch 1/30
67/67 [==============================] - 10s 117ms/step - loss: 4.1377
Epoch 2/30
67/67 [==============================] - 9s 118ms/step - loss: 2.8568
Epoch 3/30
67/67 [==============================] - 9s 118ms/step - loss: 2.3705
Epoch 4/30
67/67 [==============================] - 9s 121ms/step - loss: 2.1774
Epoch 5/30
67/67 [==============================] - 9s 122ms/step - loss: 2.0445
Epoch 6/30
67/67 [==============================] - 9s 121ms/step - loss: 1.9200
Epoch 7/30
67/67 [==============================] - 9s 122ms/step - loss: 1.8076
Epoch 8/30
67/67 [==============================] - 9s 122ms/step - loss: 1.7040
Epoch 9/30
67/67 [==============================] - 9s 123ms/step - loss: 1.6234
Epoch 10/30
67/67 [==============================] - 9s 123ms/step - loss: 1.5546
Epoch 11/30
67/67 [==============================] - 9s 124ms/step - loss: 1.4968
Epoch 12/30
67/67 [==============================] - 9s 127ms/step - loss: 1.4440
Epoch 13/30
67/67 [=====

Save the model!

In [60]:
model.save('shakespeare_gen.h5')

## Step 6: Generating text

Currently our model only expects 128 sequences at a time. We can create a new model that only expects a batch_size=1. We can create a new model with this batch size, then load our saved models weights. Then call .build() on the model:

In [61]:
from tensorflow.keras.models import load_model

In [62]:
model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('shakespeare_gen.h5')

model.build(tf.TensorShape([1, None]))

In [63]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             5248      
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1026)           3361176   
_________________________________________________________________
dense_1 (Dense)              (1, None, 82)             84214     
Total params: 3,450,638
Trainable params: 3,450,638
Non-trainable params: 0
_________________________________________________________________


In [68]:
def generate_text(model,start_seed,gen_size=500,temp=1.0):
  '''
  model: Trained Model to Generate Text
  start_seed: Intial Seed text in string form
  gen_size: Number of characters to generate

  Basic idea behind this function is to take in some seed text, format it so
  that it is in the correct shape for our network, then loop the sequence as
  we keep adding our own predicted characters. Similar to our work in the RNN
  time series problems.
  '''

  # Number of characters to generate
  num_generate = gen_size

  # Vectorizing starting seed text
  input_eval = [char_to_ind[s] for s in start_seed]

  # Expand to match batch format shape
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty list to hold resulting generated text
  text_generated = []

  # Temperature effects randomness in our resulting text
  # The term is derived from entropy/thermodynamics.
  # The temperature is used to effect probability of next characters.
  # Higher probability == lesss surprising/ more expected
  # Lower temperature == more surprising / less expected

  temperature = temp

  # Here bacth size == 1
  model.reset_states()

  for i in range(num_generate):

    # Generate predictions
    predictions = model(input_eval)

    # Remove the batch shape dimension
    predictions = tf.squeeze(predictions, 0)

    # Use a categorical distribution to select hte next character
    predictions = predictions / temperature
    predicted_id = tf.random.categorical(predictions, num_samples = 1)[-1,0].numpy()

    # Pass the predicted character for the next input
    input_eval = tf.expand_dims([predicted_id],0)

    # Transform back to character letter
    text_generated.append(ind_to_char[predicted_id])
  
  return (start_seed + ''.join(text_generated))

In [73]:
print(generate_text(model, "JULIET", gen_size=1000, temp=1.0))

JULIETUPUDESPUSTULUTEUNENUSUSw're blastly
    Oft been appearing to you.
  CORIOLANUS. So- spake quitencr. What?
  MENENIUS. O that craver you for you. 'Tis strange.
  THIRD LORD. Dut him awaking your cimble wish.
ADRIANA. choice you so?
  CELIA. And this will grant itay, that know her got.
  When I gate of pull him tar'd more
  As the roys-muquit by your swords,
    Though there first in Antoniams; till which I,
    Their kings from fancy relieity. I had without your
    flait. You govers it not; sir, dirmatic; how does live.
    Have an eniurgers-
  CORIOLANUS. Go, poor lady.
  It is the court! I have spoke of, at first mistress; say,
  And that mine eye of nest. How do't! this geary villain:
    For it she hast never ke ut thee, and trums will swoon?
  No more comes on. Why should think you.
  High office of death
  The starging do as I do vewill does soo last,
  His bander in the school'd at the womb's silence?
  CYMBELINE. Will you go?  
    And I fertune born of virginity is near